In [1]:
import pandas as pd
import sys
import os

module_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from natural_language_inference.snli_service import get_model_summary

In [3]:
claude_3_5_haiku_df = pd.read_csv('../sentiment_analysis/predictions_train_claude-3-5-haiku.csv')
gemini_2_0_flash_df = pd.read_csv('../sentiment_analysis/predictions_train_gemini-2.0-flash.csv')
gpt_4o_mini_df = pd.read_csv('../sentiment_analysis/predictions_train_gpt-4o-mini.csv')
vader_df = pd.read_csv('../data/amdb_dataset/train/train_combined_vader.csv')

In [4]:
SAMPLES_PER_CLASS = 180

small_balanced_df = vader_df.groupby('mark').sample(n=SAMPLES_PER_CLASS, random_state=42)
small_balanced_df = small_balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

display(small_balanced_df['mark'].value_counts())
display(len(small_balanced_df['mark']))

vader_df = small_balanced_df.copy()

mark
1     180
4     180
8     180
10    180
9     180
2     180
7     180
3     180
Name: count, dtype: int64

1440

In [5]:
display(vader_df.head(1))
display(gpt_4o_mini_df.head(1))
display(gemini_2_0_flash_df.head(1))
display(claude_3_5_haiku_df.head(1))

,Unnamed: 0,text,mark,label,prediction
0,3319,There are some bad movies out there. Most of t...,1,0,0


,text,mark,label,pred_raw,pred_label,time_sec,prompt_tokens,completion_tokens,total_tokens
0,Despite the high ratings given to this film by...,2,0,Negative,0,0.702807,142,1,143


,text,mark,label,pred_raw,pred_label,time_sec,prompt_tokens,completion_tokens,total_tokens
0,Despite the high ratings given to this film by...,2,0,Negative,0,0.512161,133,2,135


,text,mark,label,pred_raw,pred_label,time_sec,prompt_tokens,completion_tokens,total_tokens
0,Despite the high ratings given to this film by...,2,0,Negative,0,1.183462,155,5,160


In [6]:
results_list = []

results_list.append(get_model_summary(claude_3_5_haiku_df, "claude-3-5-haiku", gold_label='label', pred_label='pred_label'))
results_list.append(get_model_summary(gemini_2_0_flash_df, "gemini-2.0-flash", gold_label='label', pred_label='pred_label'))
results_list.append(get_model_summary(gpt_4o_mini_df, "gpt-4o-mini", gold_label='label', pred_label='pred_label'))
results_list.append(get_model_summary(vader_df, "vader", gold_label='label', pred_label='prediction'))

In [7]:
df_summary = pd.DataFrame(results_list)
display(df_summary.round(4))

,Model,NaN_Count,Total_Support,Accuracy,Support_Cleaned,Total Correct Predictions,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Correct_0,Correct_1
0,claude-3-5-haiku,0,1440,0.9257,1440.0,1333,0.9265,0.9257,0.9260,0.9265,0.9257,0.9260,673,660
1,gemini-2.0-flash,0,1440,0.9354,1440.0,1347,0.9362,0.9354,0.9357,0.9362,0.9354,0.9357,679,668
2,gpt-4o-mini,0,1440,0.9111,1440.0,1312,0.9263,0.9111,0.9168,0.9263,0.9111,0.9168,691,621
3,vader,0,1440,0.6812,1440.0,981,0.6997,0.6812,0.6737,0.6997,0.6812,0.6737,381,600


In [8]:
df_summary.to_csv("../sentiment_analysis/summary_all_models.csv", index=False)